# Train GW models based on PCA

Train models per class using PCA as the reduction input.

In [ ]:
import pathlib

import geopandas as gpd
import joblib
import pandas as pd

from gwlearn.ensemble import GWRandomForestClassifier
from gwlearn.linear_model import GWLogisticRegression

Prepare folders.

In [ ]:
pathlib.Path("/data/uscuni-restricted/06_models").mkdir(exist_ok=True)
pathlib.Path("/data/uscuni-restricted/06_models/pca").mkdir(exist_ok=True)

Read data.

In [ ]:
pcas = gpd.read_parquet("/data/uscuni-restricted/05_pcs/pcs_new.parquet")
clusters = pd.read_csv(
    "/data/uscuni-restricted/geometries/cluster_assignment_v3.csv",
    dtype={"kod_nadzsj_d": str},
)
cluster_mapping = pd.read_parquet(
    "/data/uscuni-ulce/processed_data/clusters/cluster_mapping_v3.pq"
)
data = pcas.merge(clusters, left_on="nadzsjd", right_on="kod_nadzsj_d")
variables = data.columns.drop(["geometry", "kod_nadzsj_d", "final_without_noise"])

mapped = data["final_without_noise"].map(cluster_mapping[3])

In [ ]:
bandwidth_dict = {1: 1500, 3: 1500, 4: 1300, 5: 1500, 6: 300, 7: 1500, 8: 900}

Loop over cluster labels, train RF and LR and save them.

In [ ]:
for label in [1, 3, 4, 5, 7, 8]:
    p = pathlib.Path(f"/data/uscuni-restricted/06_models/pca/label_{label}/")
    p.mkdir(exist_ok=True)
    y = mapped == label
    bandwidth = bandwidth_dict[label]

    # RF
    print(f"Label: {label}, training RF.")
    rf_path = p.joinpath("rf")
    rf_path.mkdir(exist_ok=True)

    rf_lm_path = rf_path.joinpath("local")
    rf_lm_path.mkdir(exist_ok=True)

    rf = GWRandomForestClassifier(
        bandwidth=bandwidth,
        fixed=False,
        n_jobs=-1,
        keep_models=False,
        batch_size=1000,
        min_proportion=0.05,
        class_weight="balanced",
        undersample=True,
        min_samples_split=4,
        min_samples_leaf=2,
        n_estimators=100,
        random_state=42,
    )
    rf.fit(
        data[variables],
        y,
        data.representative_point(),
    )
    with open(rf_path.joinpath("model.joblib"), "wb") as f:
        joblib.dump(rf, f, protocol=5)
    print(f"Label: {label}, RF trained. Balanced accuracy: {rf.balanced_accuracy_:.3f}")

    # LR
    print(f"Label: {label}, training LR.")
    lr_path = p.joinpath("lr")
    lr_path.mkdir(exist_ok=True)

    lr_lm_path = lr_path.joinpath("local")
    lr_lm_path.mkdir(exist_ok=True)

    lr = GWLogisticRegression(
        bandwidth=bandwidth,
        fixed=False,
        n_jobs=-1,
        keep_models=False,
        batch_size=1000,
        min_proportion=0.05,
        class_weight="balanced",
        max_iter=1000,
        undersample=True,
        random_state=42,
    )
    lr.fit(
        data[variables],
        y,
        data.representative_point(),
    )
    with open(lr_path.joinpath("model.joblib"), "wb") as f:
        joblib.dump(lr, f, protocol=5)
    print(f"Label: {label}, LR trained. Balanced accuracy: {lr.balanced_accuracy_:.3f}")

In [ ]:
fas = gpd.read_parquet("/data/uscuni-restricted/05_fa/fa_new25.parquet")
clusters = pd.read_csv(
    "/data/uscuni-restricted/geometries/cluster_assignment_v3.csv",
    dtype={"kod_nadzsj_d": str},
)
cluster_mapping = pd.read_parquet(
    "/data/uscuni-ulce/processed_data/clusters/cluster_mapping_v3.pq"
)
data = fas.merge(clusters, left_on="nadzsjd", right_on="kod_nadzsj_d")
variables = data.columns.drop(["geometry", "kod_nadzsj_d", "final_without_noise"])

mapped = data["final_without_noise"].map(cluster_mapping[3])

In [ ]:
bandwidth_dict = {1: 1300, 3: 1500, 4: 1700, 5: 500, 6: 300, 7: 1300, 8: 900}

In [ ]:
for label in [1, 3, 4, 5, 7, 8]:
    p = pathlib.Path(f"/data/uscuni-restricted/06_models/fa/label_{label}/")
    p.mkdir(exist_ok=True)
    y = mapped == label
    bandwidth = bandwidth_dict[label]

    # RF
    print(f"Label: {label}, training RF.")
    rf_path = p.joinpath("rf")
    rf_path.mkdir(exist_ok=True)

    rf_lm_path = rf_path.joinpath("local")
    rf_lm_path.mkdir(exist_ok=True)

    rf = GWRandomForestClassifier(
        bandwidth=bandwidth,
        fixed=False,
        n_jobs=-1,
        keep_models=rf_lm_path,
        batch_size=1000,
        min_proportion=0.05,
        class_weight="balanced",
        undersample=True,
        min_samples_split=4,
        min_samples_leaf=2,
        n_estimators=100,
        random_state=42,
    )
    rf.fit(
        data[variables],
        y,
        data.representative_point(),
    )
    with open(rf_path.joinpath("model.joblib"), "wb") as f:
        joblib.dump(rf, f, protocol=5)
    print(f"Label: {label}, RF trained. Balanced accuracy: {rf.balanced_accuracy_:.3f}")

    # LR
    print(f"Label: {label}, training LR.")
    lr_path = p.joinpath("lr")
    lr_path.mkdir(exist_ok=True)

    lr_lm_path = lr_path.joinpath("local")
    lr_lm_path.mkdir(exist_ok=True)

    lr = GWLogisticRegression(
        bandwidth=bandwidth,
        fixed=False,
        n_jobs=-1,
        keep_models=lr_lm_path,
        batch_size=1000,
        min_proportion=0.05,
        class_weight="balanced",
        max_iter=1000,
        undersample=True,
        random_state=42,
    )
    lr.fit(
        data[variables],
        y,
        data.representative_point(),
    )
    with open(lr_path.joinpath("model.joblib"), "wb") as f:
        joblib.dump(lr, f, protocol=5)
    print(f"Label: {label}, LR trained. Balanced accuracy: {lr.balanced_accuracy_:.3f}")

In [ ]:
umap_option = "umap_dim20_nb5_euclidean"

In [ ]:
umap = gpd.read_parquet("/data/uscuni-restricted/05_umap/umap_new.parquet")
clusters = pd.read_csv(
    "/data/uscuni-restricted/geometries/cluster_assignment_v3.csv",
    dtype={"kod_nadzsj_d": str},
)
cluster_mapping = pd.read_parquet(
    "/data/uscuni-ulce/processed_data/clusters/cluster_mapping_v3.pq"
)
data = umap.merge(clusters, left_on="nadzsjd", right_on="kod_nadzsj_d")
variables = data.columns.drop(["geometry", "kod_nadzsj_d", "final_without_noise"])

mapped = data["final_without_noise"].map(cluster_mapping[3])

In [ ]:
bandwidth_dict = {1: 1300, 3: 1500, 4: 1300, 5: 700, 6: 300, 7: 1300, 8: 700}

In [ ]:
for label in [1, 3, 4, 5, 7, 8]:
    p = pathlib.Path(f"/data/uscuni-restricted/06_models/{umap_option}/label_{label}/")
    p.mkdir(exist_ok=True)
    y = mapped == label
    bandwidth = bandwidth_dict[label]

    # RF
    print(f"Label: {label}, training RF.")
    rf_path = p.joinpath("rf")
    rf_path.mkdir(exist_ok=True)

    rf_lm_path = rf_path.joinpath("local")
    rf_lm_path.mkdir(exist_ok=True)

    rf = GWRandomForestClassifier(
        bandwidth=bandwidth,
        fixed=False,
        n_jobs=-1,
        keep_models=False,
        batch_size=1000,
        min_proportion=0.05,
        class_weight="balanced",
        undersample=True,
        min_samples_split=4,
        min_samples_leaf=2,
        n_estimators=100,
        random_state=42,
    )
    rf.fit(
        data[variables],
        y,
        data.representative_point(),
    )
    with open(rf_path.joinpath("model.joblib"), "wb") as f:
        joblib.dump(rf, f, protocol=5)
    print(f"Label: {label}, RF trained. Balanced accuracy: {rf.balanced_accuracy_:.3f}")

    # LR
    print(f"Label: {label}, training LR.")
    lr_path = p.joinpath("lr")
    lr_path.mkdir(exist_ok=True)

    lr_lm_path = lr_path.joinpath("local")
    lr_lm_path.mkdir(exist_ok=True)

    lr = GWLogisticRegression(
        bandwidth=bandwidth,
        fixed=False,
        n_jobs=-1,
        keep_models=lr_lm_path,
        batch_size=1000,
        min_proportion=0.05,
        class_weight="balanced",
        max_iter=1000,
        undersample=True,
        random_state=42,
    )
    lr.fit(
        data[variables],
        y,
        data.representative_point(),
    )
    with open(lr_path.joinpath("model.joblib"), "wb") as f:
        joblib.dump(lr, f, protocol=5)
    print(f"Label: {label}, LR trained. Balanced accuracy: {lr.balanced_accuracy_:.3f}")